# 课程评论与用户信息爬取

## 思路
- 想法：输出三种类型的文件
    - 第一种类型： 200个选课_v3，课程增加一列课程简介，在这个200个选课种我们其实就包含了课程的基本信息以及课程总体打分
    - 第二种类型： 每个课对应一个excel文件，专门存储每一个课程评价信息，课程评价又会包括user_id，user_score,user_thumb_up等等
    - 第三种类型： 将第二种类型的文件进行user_id进行取集合（也就是去重的操作），然后慢慢盘它，这个有个问题就是网速问题和网页喧嚷问题，只能单独拿出来盘它了。
- 讨论区暂定（还不知道以什么形式爬取，目前没想法）

In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity="all"

In [2]:
# 导入相关的库
import os
import time
import emoji
# import asyncio
# import requests
import pandas as pd
# import xlwings as xw
# from lxml import etree
# from bs4 import BeautifulSoup
from selenium import webdriver
# from scrapy.http import HtmlResponse
from selenium.webdriver.common.by import By
import re

### 定义一个类

In [4]:
class Data_collector():
    '''
    这个类的主要作用是方便后面的代码的可读性，将一些重复的操作实现模块化处理
    '''
    def __init__(self,excel_path):
        '''
        :param excel_path: 200个选课_v2的地址
        这里的作用是定义了四个属性,df_200,new_names,course200_urls,driver
        '''
        # 初始化参数
        self.df_200 = pd.read_excel(excel_path)

        # 合并一个课程名称（开课单位-开课老师）
        course_name = self.df_200['课程名称'].tolist()
        course_unit = self.df_200['开课单位'].tolist()
        course_teachers = self.df_200['开课老师'].tolist()

        self.new_names = [f"{it1}({it2}-{it3})" for it1,it2,it3 in zip(course_name,course_unit,course_teachers)]
        self.course200_urls = self.df_200['课程url'].tolist()

        self.driver = webdriver.Chrome()

    # 定义类方法
    def login(self):
        '''
        这个类方法的主要作用是进行登陆的操作
        '''
        # 首先先处理登陆的操作
        # 准备对应的url地址
        url = "https://www.icourse163.org/"

        # 下面主要先进行处理登陆的问题
        person = {
            # "account" : "13501408005" ,
            # "password": "qazwsx123520",
            "account" : "13539651927"     ,
            "password": "qazmlp123456@..."
        }

        # 首先创建一个driver实例对象
        self.driver.maximize_window()

        # 避开 chrome 检测
        self.driver.execute_cdp_cmd(
            "Page.addScriptToEvaluateOnNewDocument",
            {
                "source": """Object.defineProperty(navigator, 'webdriver', {get: () => undefined})"""
            }
        )

        # 打开对应的网站
        self.driver.get(url)

        # 登录
        self.driver.find_element(by=By.CSS_SELECTOR,value="#j-topnav > div > a").click()
        # 其他登录方式
        self.driver.find_element(by=By.CSS_SELECTOR,value="div.ux-login-set-scan-code_ft > span").click()
        # 手机号登录
        self.driver.find_element(by=By.CSS_SELECTOR,value="div.ux-tabs-underline > ul > li:nth-child(2)").click()
        self.driver.switch_to.frame(self.driver.find_element(by=By.CSS_SELECTOR,value="div.ux-login-urs-phone-wrap.f-pr > div > iframe"))

        # 输入账号
        self.driver.find_element(by=By.XPATH,value="/html/body/div[2]/div[2]/div[2]/form/div/div[2]/div[2]/input").clear()
        self.driver.find_element(by=By.XPATH,value="/html/body/div[2]/div[2]/div[2]/form/div/div[2]/div[2]/input").send_keys(person["account"])
        # 输入密码
        self.driver.find_element(by=By.XPATH,value="/html/body/div[2]/div[2]/div[2]/form/div/div[4]/div[2]/input[2]").clear()
        self.driver.find_element(by=By.XPATH,value="/html/body/div[2]/div[2]/div[2]/form/div/div[4]/div[2]/input[2]").send_keys(person["password"])

        time.sleep(1)
        # 登录
        self.driver.find_element(by=By.XPATH,value="//div[@class='m-container']/div[@class='u-tab f-cb']//following::div[@class='f-cb loginbox']").click()

        time.sleep(2)
        self.driver.implicitly_wait(5)

    def to_course_pages(self,course_name):
        '''
        这个函数主要作用时进入对应的课程的页面
        :param course_name: 表示的是课程的名称
        :return: 进入到对应的课程页面中
        '''
        # 首先先提取出对应的课程名称和课程url
        index = self.new_names.index(course_name)
        course_url = self.course200_urls[index]
        self.driver.execute_script("window.open('{}');".format(course_url))
        # 切换窗口
        self.driver.switch_to.window(self.driver.window_handles[-1])

    def scrapy_course_introduction(self):
        content_elements = self.driver.find_elements(by=By.CSS_SELECTOR,value="#content-section > div:nth-child(4) > div > p")
        content = ''.join([t.text for t in content_elements])
        # print(content)
        # print(len(content))
        return content

    def close_page(self):
        # 实现页面关闭
        self.driver.close()
        # 并且切换页面
        self.driver.switch_to.window(self.driver.window_handles[-1])

    def generate_v3(self):
        content_lst = []
        for name,url in zip(self.new_names,self.course200_urls):
            # 首先先进入到对应的课程页面，然后对页面的课程简介进行爬取
            self.to_course_pages(name)
            tp_content = self.scrapy_course_introduction()
            # 有些课特别恶心，它的课程简介是以公众号那种类型来进行展示的，所以提取到的content为空
            if len(tp_content) == 0:
                tp_content = '需要人工进行爬取'
            content_lst.append(tp_content)
            print(name)
            # 关闭这个课程的页面
            self.close_page()

        # 下面进行合成excel文件变成    200个选课_v3.excel
        new_df = self.df_200.copy()
        new_df['课程简介'] = content_lst
        new_df.to_excel('200个选课_V3.xlsx',index=False)

    def generate_type2_dict(self):
        dic = {
        "comments":[],
        "comments_thumbs_up":[],
        "comments_stars":[],
        "user_url": [],
        "user_name": []
        }
        return dic

    # 规范字符串
    def canonical_string(self,s):
        # 有一些课程搞事情，名字里面包含了一些字符，文件命名的时候不能出现冒号等一些特殊符号，这里通过正则表达式进行文本的清洗
        return re.sub(r"[\:|\\|\/|\*|\?|\"|\<|\>|\|]+","",s)

    def scrapy_comments_and_user_name(self,start,end):
        # 此处会爬的东西主要有三个:评论正文,评论点赞个数,评论打分,user_id,user的info，再加一列
        # 首先先创建文件夹
        if not os.path.exists('全部课程_comments'):
            os.makedirs('全部课程_comments')
        print(self.new_names[(start-1):(end)])
        for name,url in zip(self.new_names[(start-1):(end)],self.course200_urls[(start-1):(end)]):
            #print(name, url, sep="\t")
            excel_name = f"./全部课程_comments/{self.canonical_string(name)}.xlsx"
            # 首先先到达对应的页面
            self.to_course_pages(name)
            # 点击课程评价
            self.driver.find_element(by=By.CSS_SELECTOR,value="#review-tag-button").click()

            # 获取对应的课程字典
            tp_dict = self.generate_type2_dict()

            # 首先获取评论三列
            # 评论数
            comment_num = int(self.driver.find_element(by=By.CSS_SELECTOR,value="#review-tag-num").text[1:-1])

            # 页数
            quotient = comment_num // 20
            # 逐页点开
            for i in range(1,quotient+2):

                # 把所有需要展开的先展开
                unfold = self.driver.find_elements(by=By.CSS_SELECTOR,value="div.ux-mooc-comment-course-comment_comment-list_item_body_content > span:nth-child(2)")
                for j in unfold:
                    j.click()

                # 评论正文
                comment = self.driver.find_elements(by=By.CSS_SELECTOR,value="div.ux-mooc-comment-course-comment_comment-list_item_body_content > span")
                comment_lst = [damn1.text for damn1 in comment]
                tp_dict['comments'] = tp_dict['comments']+comment_lst

                # 点赞数
                thumbs_up = self.driver.find_elements(by=By.CSS_SELECTOR,value="div.ux-mooc-comment-course-comment_comment-list_item_body_comment-info_actions > span > span > span:nth-child(2)")
                thumbs_up_lst = [damn2.text for damn2 in thumbs_up]
                tp_dict['comments_thumbs_up'] = tp_dict['comments_thumbs_up']+thumbs_up_lst

                # 给星 **那个位置** 的对象
                stars = self.driver.find_elements(by=By.CSS_SELECTOR,value="div.ux-mooc-comment-course-comment_comment-list_item_body_user-info > span > div > div.star-point")

                # 空列表用来存储星星数量
                star = []
                for st in stars:
                    star.append(len(st.find_elements(by=By.CSS_SELECTOR,value="i")))
                tp_dict['comments_stars'] = tp_dict['comments_stars'] + star

                users_urls = [i.get_attribute("href") for i in self.driver.find_elements(by=By.CSS_SELECTOR,value="div.ux-mooc-comment-course-comment_comment-list_item_body_user-info > a")]
                users_names = [i.text for i in self.driver.find_elements(by=By.CSS_SELECTOR,value="div.ux-mooc-comment-course-comment_comment-list_item_body_user-info > a")]

                tp_dict['user_url'] = tp_dict['user_url'] + users_urls
                tp_dict['user_name'] = tp_dict['user_name'] + users_names

                # 如果是最后一页,就停止,不用再点了
                if i != quotient+1:
                    # 如果还不是最后一页,就要点击下一页
                    self.driver.find_element(by=By.CSS_SELECTOR,value="li.ux-pager_btn.ux-pager_btn__next > a").click()
            return pd.DataFrame(tp_dict),tp_dict
            # pd.DataFrame(tp_dict).to_excel(excel_name,index=False)
            # self.close_page()


    def extract_user_url(self):
        # 这个类方法将会实现提取出不重复的user_id,并且写到一个excel文件中
        all_course_comments_lst = [f"全部课程_comments/{damn1}" for damn1 in os.listdir('全部课程_comments') if damn1[-4:] == 'xlsx']
        df_all = pd.DataFrame()
        for item in all_course_comments_lst:
            tp_df = pd.read_excel(item)
            df_all = pd.concat([df_all,tp_df])

        wow_df = df_all.loc[:,['user_url','user_name']].drop_duplicates()
        wow_df.to_excel('用户信息v1.xlsx',index=False)


    def generate_user_info_excel(self):
        # 这个类方法将会在上面输出的excel文件中不断进行写入
        pass


###  处理第一种类型文件：200个选课_v3（下面这个我已经跑完了，不用再跑了）
- 这里想要做的是在v2版本上加多一列课程简介
- 后面有些需要重新人工进行爬取，后面会升级为200个选课_v4.xlsx

In [25]:
# 创建实例对象
my_collector = Data_collector('200个选课_V2.xlsx')
my_collector.login()
my_collector.generate_v3()

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"/html/body/div[2]/div[2]/div[2]/form/div/div[2]/div[2]/input"}
  (Session info: chrome=103.0.5060.134)
Stacktrace:
0   chromedriver                        0x0000000104b2dd14 chromedriver + 3792148
1   chromedriver                        0x0000000104ac4828 chromedriver + 3360808
2   chromedriver                        0x00000001047c50b8 chromedriver + 217272
3   chromedriver                        0x00000001047f4e90 chromedriver + 413328
4   chromedriver                        0x000000010481da60 chromedriver + 580192
5   chromedriver                        0x00000001047ea318 chromedriver + 369432
6   chromedriver                        0x0000000104b031e8 chromedriver + 3617256
7   chromedriver                        0x0000000104b07678 chromedriver + 3634808
8   chromedriver                        0x0000000104b0bc6c chromedriver + 3652716
9   chromedriver                        0x000

### 处理第二种类型的文件：每一个课程会输出一个excel文件

In [6]:
my_collector = Data_collector('200个选课_V4.xlsx')
my_collector.login()
# start 表示的是起始的index end是终止的index
# 1,1 表示第一个课 i,i 表示第i个课 1,2表示第一和第二个课
df, dict1 = my_collector.scrapy_comments_and_user_name(157,157)

['走进创业(南京大学-王自强/陶向南)']


In [8]:
df.to_csv('走进创业(南京大学-王自强陶向南).csv',index=False)

In [12]:
df.to_csv('Python语言程序设计(北京理工大学-嵩天).csv',index=False)

In [15]:
my_collector.new_names[189]
all_course_comments_lst = [f"全部课程_comments/{damn1}" for damn1 in os.listdir('全部课程_comments') if damn1[-4:] == 'xlsx' or damn1[-3:] == 'csv']
len(all_course_comments_lst)

200

In [4]:
all_course_comments_lst = [f"全部课程_comments/{damn1}" for damn1 in os.listdir('全部课程_comments') if damn1[-4:] == 'xlsx' or damn1[-3:] == 'csv']
df_all = pd.DataFrame()
for item in all_course_comments_lst:
    if item[-4:] == 'xlsx':
        tp_df = pd.read_excel(item)
    elif item[-3:] == 'csv':
        tp_df = pd.read_csv(item)
    df_all = pd.concat([df_all,tp_df])

In [5]:
small_df = df_all.loc[:,['user_url','user_name']].drop_duplicates()

In [6]:
small_df

,user_url,user_name
0,https://www.icourse163.org/home.htm?userId=138...,GreatMarketLeader
1,https://www.icourse163.org/home.htm?userId=115...,杨立宇mooc
2,https://www.icourse163.org/home.htm?userId=103...,席颜
3,https://www.icourse163.org/home.htm?userId=101...,Full_stack
4,https://www.icourse163.org/home.htm?userId=103...,今天美拉学习了吗
...,...,...
114,https://www.icourse163.org/home.htm?userId=139...,神经蛙mooc243
115,https://www.icourse163.org/home.htm?userId=139...,爱吃肉的小棉袄
116,https://www.icourse163.org/home.htm?userId=139...,小朋友mooc495
117,https://www.icourse163.org/home.htm?userId=139...,DXmooc149


In [7]:
small_df.to_csv('all_users_info_pre.csv',index = False)

In [12]:
small_df.iloc[0,:]['user_url']

'https://www.icourse163.org/home.htm?userId=1385226274'

In [26]:
class Damn():
    def __init__(self,excel_name):
        # 这个类有四个属性，第一个属性为user_info_df,第二个属性为tag的位置,第三个属性为所有用户的数量，第四个属性为driver
        self.user_info_df = pd.read_csv(excel_name)
        self.tag = 0
        self.all_members_num = self.user_info_df.shape[0]
        self.driver = webdriver.Chrome()

        self.driver.maximize_window()

    def create_txt_file(self):
        with open('users_info.txt','a+',encoding='utf-8') as f:
            f.write("{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\r".format("user_name","user_url","enrolled_courses_num","enrolled_courses","follow_num","fans_num","theme_n_reply","thumbs_up","studying_minute","certificate_of_excellence_num","certificate_of_excellence","certificate_of_competency_num","certificate_of_competency"))

    def close_page(self):
        # 实现页面关闭
        self.driver.close()
        # 并且切换页面
        self.driver.switch_to.window(self.driver.window_handles[-1])

    def to_user_page(self,user_url):
        # 在新窗口打开
        self.driver.execute_script("window.open('{}');".format(user_url))
        # 进行切换窗口
        self.driver.switch_to.window(self.driver.window_handles[-1])

    def parse(self):
        # 首先我们先进入到历史记录的tag以及对应的dataframe
        need_df = self.user_info_df.iloc[self.tag:,:]
        user_urls = need_df['user_url'].tolist()
        user_names = need_df['user_name'].tolist()

        for user_url,user_name in zip(user_urls,user_names):
            try:
                # 先进入对应的url中
                self.to_user_page(user_url)

                # 下面将会寻找对应的元素
                # 已修读课程数量
                enrolled_courses_num = self.driver.find_element(by=By.CSS_SELECTOR, value="#j-module-tab > div > div > a > span > i").text
                if len(enrolled_courses_num) == 0:
                    enrolled_courses_num = 0
                    enrolled_courses = []
                else:
                    # 已修读课程名称（其实没有必要全部取出，至多取出第一页即可）
                    enrolled_courses = ", ".join([i.text for i in self.driver.find_elements(by=By.CSS_SELECTOR, value="#j-coursewrap > div > div.u-cc-courseFunc.f-cb > div > div > a > div.u-cc-courseFunc_div32 > span")])


                # 关注人数
                follow_num = self.driver.find_element(by=By.CSS_SELECTOR, value="#j-self-content > div > div.u-ui-f2f > span > span:nth-child(1)").text[2:-1]

                # 粉丝量
                fans_num = self.driver.find_element(by=By.CSS_SELECTOR, value="#j-self-content > div > div.u-ui-f2f > span > span:nth-child(2)").text[2:-1]

                # 主题/回复
                theme_n_reply = self.driver.find_element(by=By.CSS_SELECTOR, value="#j-self-content > div > div.u-ui-box.f-fr > div > div > div.u-ui-discuss-cont > span").text

                # 获赞数量
                thumbs_up = self.driver.find_element(by=By.CSS_SELECTOR, value="#j-self-content > div > div.u-ui-box.f-fr > div > div > div.u-ui-zan-cnt > span").text

                # 学习时长
                studying_duration = self.driver.find_element(by=By.CSS_SELECTOR, value="#j-self-content > div > div.u-ui-box.f-fr > div > div > div.u-ui-time-cont > span").text
                if "时" in studying_duration:
                    ch1 = studying_duration.index("时")
                    # 转换成分钟
                    studying_minute = int(studying_duration[:ch1])*60 + int(studying_duration[ch1+1:-1])
                else:
                    studying_minute = int(studying_duration[:-1])

                # 优秀证书
                self.driver.get("{}{}{}{}".format(user_url[:-6],"mycert?",user_url.split("?")[1][:-13],"&type=1&p=1"))
                time.sleep(1)
                certificate_of_excellence = [i.text for i in self.driver.find_elements(by=By.CSS_SELECTOR, value="div.u-certCard-courseFunc_card_title.f-thide")]

                # 合格证书
                self.driver.get("{}{}{}{}".format(user_url[:-6],"mycert?",user_url.split("?")[1][:-13],"&type=2&p=1"))
                time.sleep(1)
                certificate_of_competency = [i.text for i in self.driver.find_elements(by=By.CSS_SELECTOR, value="div.u-certCard-courseFunc_card_title.f-thide")]

                # 上面表示的是已经提取到对应的信息了，下面将提取到的东西写到文件中
                want_to_write = "{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\t{}\r".format(user_name,user_url,int(enrolled_courses_num),", ".join(enrolled_courses),int(follow_num),int(fans_num),int(theme_n_reply),int(thumbs_up),studying_minute,len(certificate_of_excellence),", ".join(certificate_of_excellence),len(certificate_of_competency),", ".join(certificate_of_competency))

                with open('users_info.txt','a+',mode='utf-8') as f:
                    f.write(want_to_write)

                self.tag += 1 # 记录已经写到哪里了
                self.close_page()
            except:
                self.close_page()
                break



In [27]:
bianqiang = Damn("all_users_info_pre.csv")

In [29]:
# 首先先创建一个新的文件存储
bianqiang.create_txt_file()

In [32]:
bianqiang.parse()

In [25]:
bianqiang.tag

0

In [33]:
os.remove('users_info.txt')